# Model run examples

**TASK 2.1. Write an example code based on the model description and example
parameters listed in the supplementary information of Allison et al. 2010.**

The model is implemented in the python module called "model.py", following the description provided in task 1.2

This showcases how the python implementation can be run on different type of inputs

In [22]:
from model import (
    ModelRunner,
)
import pandas as pd
import plotly_express as px
from utils import plot_model_history
from datetime import datetime as dt, timedelta


In [50]:
# Running with single dtemp /dt

# initial sysvar are taken from  Allison et al 2010. Supp mat.
initial_sysvar = {
    "soc": 100,
    "doc": .5,
    "mic": .5,
    "temp": 20,
    "enz": 0.01,
    "co2":0
}

my_model = ModelRunner(initial_sysvar)

my_model.compute_model(3, 0.0005, 0.0005)


computing model for dtemp=3, input_soc=0.0005,input_doc=0.0005 
inital system state: self.sysvar={'soc': 100, 'doc': 0.5, 'mic': 0.5, 'temp': 20, 'enz': 0.01, 'co2': 0}
computation finished
current state of the system:self.sysvar={'soc': 99.99983980008221, 'doc': 0.34832051882970866, 'mic': 0.5399703164450745, 'temp': 23, 'enz': 0.0099925, 'co2': 0.11287686464299633}


Computing the simulation a 100 times, providing input with dataframe

In [3]:
# Ths is the input dataframe

input = pd.DataFrame([{"dtemp": 0.1, "input_soc": 0.0005, "input_doc": 0.0005, }]*100)

print("input dataframe:")
print(input.head())

# run simulation over dataframe
my_model=ModelRunner(initial_sysvar)
for i,n in input.iterrows():
    my_model.compute_model(**n.to_dict(), verbose=False)

# extract and plot  dataframe
history_df = pd.DataFrame(my_model.history).reset_index()
print("\n\noutput dataframe:")
print(history_df.head())
plot_model_history(history_df)


input dataframe:
   dtemp  input_soc  input_doc
0    0.1     0.0005     0.0005
1    0.1     0.0005     0.0005
2    0.1     0.0005     0.0005
3    0.1     0.0005     0.0005
4    0.1     0.0005     0.0005


output dataframe:
   index        soc       doc       mic  temp       enz       co2
0      0  99.999680  0.208096  0.576707  23.1  0.009985  0.217532
1      1  99.999520  0.095084  0.606169  23.2  0.009978  0.302249
2      2  99.999360  0.026672  0.623972  23.3  0.009971  0.354025
3      3  99.999198  0.003926  0.629990  23.4  0.009964  0.371922
4      4  99.999033  0.001400  0.630833  23.5  0.009957  0.374777


# task 2.2 and 2.3
Provide a visualization (publication quality) on how the SOC of a field
nearby your place
Find the meteorological data of that field you chose in subtask 2.2. Using the model of Allison et al. 2010, make predictions on how the SOC varied over a period of 1 week or 1 year under the assumptions of your choice.

To get SOC values for a location nearbye we rely on Google Earth Engine Catalog


In particular:    
- `OpenLandMap Soil Organic Carbon Content` provides current carbon estimates in Soil [link](https://developers.google.com/earth-engine/datasets/catalog/OpenLandMap_SOL_SOL_ORGANIC-CARBON_USDA-6A1C_M_v02)    
- `MOD11A1.061 Terra Land Surface Temperature` provides current surface 
temperature [link](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1)    
- `NASA Earth Exchange Global Daily Downscaled Climate Projections` to get future temperature estimates [link](https://developers.google.com/earth-engine/datasets/catalog/NASA_GDDP-CMIP6)


In [4]:
import ee

In [2]:

import ee
def authenticate_ee(
        service_account = 'digitsoiltemp@deep-bolt-386711.iam.gserviceaccount.com',
        path_to_keys = 'deep-bolt-386711-f24058c11291.json'
):
    credentials = ee.ServiceAccountCredentials(service_account, path_to_keys )
    ee.Initialize(credentials)

#-- 
authenticate_ee()

In [48]:
selected_point=(7.581966, 47.536962)


def sample_image(coords:tuple, image_id:str, band_name:str):
    # Authenticate to GEE
    authenticate_ee()

    # Define the point of interest
    point = ee.Geometry.Point(*coords)

    # Load the image you want to sample
    image = ee.Image(image_id)

    # Sample the image at the point of interest
    value = image.sample(point, 30).first().get(band_name).getInfo()

    return value

# Load the image collection and get the most recent image
def sample_collection(coords, collection_id, band_name:str, start_date:str, end_date:str):
    # Authenticate to GEE
    authenticate_ee()

    # Define the point of interest
    point = ee.Geometry.Point(*coords)

    # Load the image collection and filter it by date
    collection = ee.ImageCollection(collection_id) \
        .filterDate(start_date, end_date) \
        .sort('system:time_start')

    # Get the most recent image in the collection
    image = ee.Image(collection.first())

    # Sample the image at the point of interest
    value = image.sample(point, 30).first().get(band_name).getInfo()

    return value

# get SOC values
def get_current_soc_value(selected_point: tuple)->str:
    soc_value = sample_image(selected_point, "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02", "b10")
    #convert from 5*g/kg to mg/cm³. Average density of soil: 2.65 g/cm³
    soc_value_translated = soc_value /5 * 1000 /2.65
    return soc_value_translated

def get_soil_temp(selected_point):
    now = dt.now()

    # Calculate the date 20 days ago
    start = now - timedelta(days=20)

    temp_value = sample_collection(selected_point, 'MODIS/061/MOD11A2', 'LST_Day_1km', start.strftime('%Y-%m-%d'),now.strftime('%Y-%m-%d') )
    # convert from 8 bit kelvin to °C
    temp_celsius = temp_value * 0.02 - 283.15
    return temp_celsius

def get_temp_projection(selected_point):
    now = dt.now()

    # Calculate the date 5 days ago
    future = now + timedelta(days=365)

    temp_value = sample_collection(selected_point, "NASA/GDDP-CMIP6", 'tasmax', now.strftime('%Y-%m-%d'),future.strftime('%Y-%m-%d') )
    # convert from 8 bit kelvin to °C
    temp_celsius = temp_value - 273.15
    return temp_celsius
#--
print("current soc level:", get_current_soc_value(selected_point))
print("current soil temperature:", get_soil_temp(selected_point)) 
print("future soil temperature in 2 months:", get_temp_projection(selected_point))

current soc level: 301.8867924528302
current soil temperature: 13.430000000000007
future soil temperature in 2 months: 15.539147949218773


In [54]:
# Running model on obtained predictions

def run_model_on_predictions(location, num_days):
    initial_sysvar = {
    "soc": 100,
    "doc": .5,
    "mic": .5,
    "temp": 20,
    "enz": 0.01,
    "co2":0
}

    my_model=ModelRunner(initial_sysvar)
    # get data from api
    soc_value = get_current_soc_value(location)
    soil_temp = get_soil_temp(location)
    future_soil_temp = get_temp_projection(location)
    delta_t = future_soil_temp - soil_temp

    # set initial soc and temp
    my_model.sysvar["soc"]= soc_value
    my_model.sysvar["temp"]= soil_temp
    # prepare dataframe
    num_days= num_days
    input_df = pd.DataFrame([{"dtemp": delta_t/num_days, "input_soc": 0.000001, "input_doc": 0.000001, }]*num_days)
    #run_computation
    for i,n in input_df.iterrows():
        my_model.compute_model(**n.to_dict(), verbose=False)

    fig=plot_model_history(pd.DataFrame(my_model.history).reset_index())
    return fig
#---
selected_point=(7.581966, 47.536962)
run_model_on_predictions(selected_point, 365)

   